<a href="https://colab.research.google.com/github/adamrhurley/FYP/blob/main/MusicRecommenderSystem/MF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Reference: https://beckernick.github.io/matrix-factorization-recommender/

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving artists.csv to artists.csv
Saving userArtists.csv to userArtists.csv


In [ ]:
import io
userArtists = pd.read_csv(io.BytesIO(uploaded['userArtists.csv']))
artists = pd.read_csv(io.BytesIO(uploaded['artists.csv']))

In [ ]:
userArtists.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [ ]:
artists.head()

,artistID,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


In [ ]:
data = pd.merge(userArtists, artists, on='artistID')

In [ ]:
data.head()

,userID,artistID,weight,name,url,pictureURL
0,2,51,13883,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
1,4,51,228,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
2,27,51,85,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
3,28,51,10,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
4,62,51,528,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg


In [ ]:
dataPivoted = userArtists.pivot(index = 'userID', columns ='artistID', values = 'weight').fillna(0)
dataPivoted.head()

artistID,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,31,32,34,36,37,38,39,40,41,43,44,45,...,18696,18697,18698,18706,18707,18708,18709,18710,18711,18712,18713,18714,18715,18716,18717,18718,18719,18720,18721,18722,18723,18724,18725,18726,18727,18728,18729,18730,18734,18735,18736,18737,18738,18739,18740,18741,18742,18743,18744,18745
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
numPivot = dataPivoted.to_numpy()
meanPlays = np.mean(numPivot, axis = 1)
reshapedData = numPivot - meanPlays.reshape(-1, 1)

In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(reshapedData, k = 50)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
allPredictedPlays = np.dot(np.dot(U, sigma), Vt) + meanPlays.reshape(-1, 1)

In [ ]:
predictions = pd.DataFrame(allPredictedPlays, columns = dataPivoted.columns)
predictions.head()

artistID,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,31,32,34,36,37,38,39,40,41,43,44,45,...,18696,18697,18698,18706,18707,18708,18709,18710,18711,18712,18713,18714,18715,18716,18717,18718,18719,18720,18721,18722,18723,18724,18725,18726,18727,18728,18729,18730,18734,18735,18736,18737,18738,18739,18740,18741,18742,18743,18744,18745
0,3.662287,6.031321,3.801433,3.885595,3.757917,0.605801,103.699931,3.231338,2.133174,3.922073,4.704273,1.244612,1.484064,-1.426351,4.196748,3.507663,4.701946,-0.953035,3.810690,3.809277,3.812082,1.739513,3.811541,-5.548606,2.393979,-6.380108,3.751578,62.275409,3.932883,-7.630863,3.055360,-4.635927,4.514772,3.822984,-13.059092,7.796617,6.053978,3.789268,3.813179,-11.091711,...,3.812264,3.812264,5.290954,3.810133,3.811488,3.811539,3.811568,3.811693,3.811696,3.811725,3.811761,3.811786,3.811863,3.811882,3.811885,3.811885,3.811930,3.811950,3.811962,3.811966,3.812030,3.812039,3.802017,3.807733,3.808276,3.808493,3.808507,3.808738,3.985216,3.962263,3.959582,3.906134,3.880667,3.875808,3.865923,3.862740,3.861567,3.860394,3.860227,3.968843
1,1.179475,1.181821,1.179390,1.179072,1.180548,1.171197,0.509320,1.142359,1.174457,1.188696,1.176788,1.185913,1.193325,1.135239,1.179688,1.191550,1.181256,1.257491,1.179348,1.179377,1.179352,1.171672,1.179356,1.141396,1.194250,1.279867,1.179433,1.053802,1.192749,1.322527,1.172485,1.288623,1.178601,1.179450,1.254984,1.188990,1.180444,1.179472,1.179355,1.122233,...,1.179351,1.179351,1.186896,1.179392,1.179367,1.179366,1.179365,1.179363,1.179363,1.179362,1.179361,1.179361,1.179359,1.179359,1.179359,1.179359,1.179358,1.179358,1.179358,1.179357,1.179356,1.179356,1.179434,1.179388,1.179384,1.179382,1.179382,1.179380,1.179948,1.179869,1.179860,1.179675,1.179587,1.179570,1.179536,1.179525,1.179521,1.179517,1.179516,1.180229
2,0.891694,0.919519,0.701617,0.710352,0.794057,0.825344,16.935189,0.809258,3.400045,1.691389,0.683374,1.278264,1.969931,1.925182,0.738749,1.587012,0.944361,4.438902,0.701385,0.706936,0.701339,0.686075,0.702140,2.019032,1.774434,5.588014,0.702899,21.980389,1.744728,8.530063,0.646280,6.306425,0.776581,0.705609,8.697778,2.006306,0.777254,0.704012,0.702011,4.550698,...,0.701358,0.701358,1.425504,0.702164,0.701660,0.701641,0.701630,0.701584,0.701583,0.701572,0.701559,0.701549,0.701521,0.701513,0.701512,0.701512,0.701496,0.701488,0.701484,0.701482,0.701459,0.701455,0.702003,0.701643,0.701609,0.701595,0.701595,0.701580,0.725481,0.722278,0.721904,0.714447,0.710893,0.710215,0.708836,0.708392,0.708228,0.708064,0.708041,0.872522
3,0.120354,0.398245,0.104811,0.130828,0.266711,0.668351,15.339577,0.278009,4.012523,2.493170,0.066547,1.937799,3.194529,12.123633,0.141136,2.235236,0.456124,7.165507,0.103509,0.105937,0.103385,0.198149,0.104384,5.562949,2.649840,9.669322,0.111604,10.614525,2.564072,14.882343,0.001609,10.886477,0.103286,0.112308,12.405341,3.246858,0.127237,0.108716,0.104168,11.017200,...,0.103413,0.103413,1.632256,0.104708,0.103898,0.103867,0.103850,0.103776,0.103774,0.103756,0.103735,0.103720,0.103674,0.103663,0.103661,0.103661,0.103634,0.103622,0.103615,0.103613,0.103575,0.103569,0.105454,0.104318,0.104210,0.104167,0.104164,0.104118,0.138170,0.133556,0.133017,0.122271,0.117151,0.116174,0.114187,0.113547,0.113311,0.113075,0.113042,0.453506
4,0.049463,0.045931,0.049044,0.049238,0.048905,0.046933,0.118466,0.048436,0.042127,0.045469,0.048608,0.042271,0.044476,0.054634,0.049049,0.045140,0.048706,0.034686,0.049079,0.049072,0.049080,0.048312,0.049076,0.040320,0.045690,0.030917,0.048943,0.060170,0.045472,0.023227,0.049214,0.026331,0.049095,0.049047,-0.001027,0.046004,0.049362,0.049037,0.049078,0.045332,...,0.049080,0.049080,0.049988,0.049065,0.049074,0.049075,0.049075,0.049076,0.049076,0.049076,0.049076,0.049077,0.049077,0.049077,0.049077,0.049077,0.049078,0.049078,0.049078,0.049078,0.049078,0.049078,0.049021,0.049054,0.049057,0.049058,0.049058,0.049060,0.048785,0.048824,0.048829,0.048920,0.048963,0.048972,0.048989,0.048994,0.048996,0.048998,0

In [ ]:
def recommendations(predictions, userID, artists, userArtists, num_recommendations):
    rowNumber = userID - 1
    predictionsSorted = predictions.iloc[rowNumber].sort_values(ascending=False)
    
    user_data = userArtists[userArtists.userID == (userID)]
    userHistory = (user_data.merge(artists, how = 'left', left_on = 'artistID', right_on = 'artistID').sort_values(['weight'], ascending=False))
    recommendations = (artists[~artists['artistID'].isin(userHistory['artistID'])].
         merge(pd.DataFrame(predictionsSorted).reset_index(), how = 'left', left_on = 'artistID', right_on = 'artistID').
         rename(columns = {rowNumber: 'Predictions'}).
         sort_values('Predictions', ascending = False).iloc[:num_recommendations, :-1])

    return userHistory, recommendations

In [ ]:
alreadyListened, predictions = recommendations(predictions, 43, artists, userArtists, 5)

In [ ]:
alreadyListened.head()

,userID,artistID,weight,name,url,pictureURL
3,43,64,6390,RÃ¶yksopp,http://www.last.fm/music/R%C3%B6yksopp,http://userserve-ak.last.fm/serve/252/10953763...
27,43,748,3115,Boards of Canada,http://www.last.fm/music/Boards+of+Canada,http://userserve-ak.last.fm/serve/252/37989495...
1,43,56,2691,Daft Punk,http://www.last.fm/music/Daft+Punk,http://userserve-ak.last.fm/serve/252/10923145...
11,43,154,1561,Radiohead,http://www.last.fm/music/Radiohead,http://userserve-ak.last.fm/serve/252/8461967.jpg
32,43,1099,1227,The Prodigy,http://www.last.fm/music/The+Prodigy,http://userserve-ak.last.fm/serve/252/24606465...


In [ ]:
predictions

,artistID,name,url,pictureURL
263,288,Rihanna,http://www.last.fm/music/Rihanna,http://userserve-ak.last.fm/serve/252/53023109...
431,461,Miley Cyrus,http://www.last.fm/music/Miley+Cyrus,http://userserve-ak.last.fm/serve/252/58452885...
668,701,Shakira,http://www.last.fm/music/Shakira,http://userserve-ak.last.fm/serve/252/52116105...
275,300,Katy Perry,http://www.last.fm/music/Katy+Perry,http://userserve-ak.last.fm/serve/252/42128121...
436,466,Ke$ha,http://www.last.fm/music/Ke%24ha,http://userserve-ak.last.fm/serve/252/47829587...
